In [5]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
import numpy as np
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pytz
import pandas as pd
import cufflinks as cf
from django.db.models import Q
from apps.environment_simulator.sevice_layer.market_simulator import MarketSimulator
from apps.environment_simulator.models import SimulatedStockBuffer
import gymnasium as gym
import ray
from gymnasium import spaces
from ray.rllib.algorithms import ppo
from os import path, environ
from gymnasium.envs.registration import register
from ray import tune

In [2]:
db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}

In [3]:


ray.shutdown()
ray.init(runtime_env={"working_dir": "/Users/mirbilal/Desktop/MobCommission/commissionV2/"})

the_current_time_step  = datetime(year=1995, month=1, day=1, hour=10)

config = {
        "env": MarketSimulator,
        "env_config": {
            "db_params": db_params, 
            "max_episode_steps": 150, 
            "the_current_time_step": the_current_time_step,
            "print_output": False
        },
        "num_workers": 4,
        "entropy_coeff": 0.5,
        "framework_str": "tf2"
}

algo = ppo.PPO(config=config)

2023-10-08 19:53:20,574	INFO worker.py:1642 -- Started a local Ray instance.
2023-10-08 19:53:21,027	INFO packaging.py:518 -- Creating a file package for local directory '/Users/mirbilal/Desktop/MobCommission/commissionV2/'.
2023-10-08 19:53:21,316	INFO packaging.py:346 -- Pushing file package 'gcs://_ray_pkg_b18693a824f40c39.zip' (8.25MiB) to Ray cluster...
2023-10-08 19:53:21,355	INFO packaging.py:359 -- Successfully pushed file package 'gcs://_ray_pkg_b18693a824f40c39.zip'.
2023-10-08 19:53:24,363	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
/Users/mirbilal/Desktop/MobCommission/commissionV2/v2env/lib/python3.10/site-pack

In [14]:
# mean_ppo = []
# # for _ in range(20):

# #     result = algo.train()
# #     print("episode reward mean:", _, result['episode_reward_mean'])
# #     mean_ppo.append(result['episode_reward_mean'])

# # for _ in range(20):
# tune.run(
#     "PPO",
#     config=config,
#     stop={"training_iteration": 1},
#     verbose=1,
#     local_dir="/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment_simulator/logs/" 
# )
# # result = analysis.trials[0].last_result
# # # print("episode reward mean:", _, result['episode_reward_mean'])
# # mean_ppo.append(result['episode_reward_mean'])

mean_ppo = []
for _ in range(20):

    result = algo.train()
    print("episode reward mean:", _, result['episode_reward_mean'])
    mean_ppo.append(result['episode_reward_mean'])


2023-10-08 20:08:20,365	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=31018) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=31018) 2023-10-08 20:08:34,093	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config=

TuneError: ('Trials did not complete', [PPO_MarketSimulator_6d425_00000])

In [56]:
checkpoint_path = "/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment_simulator/simulated_checkpoints/"

algo.save(checkpoint_dir=checkpoint_path)

TrainingResult(checkpoint=Checkpoint(filesystem=local, path=/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment_simulator/simulated_checkpoints/), metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'__all__': {'num_agent_steps_trained': 128.0, 'num_env_steps_trained': 4000.0, 'total_loss': 10.009098821611547}, 'default_policy': {'total_loss': 10.009098821611547, 'policy_loss': -0.007832370704409283, 'vf_loss': 10.0, 'vf_loss_unclipped': 218934049.85927504, 'vf_explained_var': -7.670329832064825e-05, 'entropy': 8.125155032570682, 'mean_kl_loss': 0.011148096544407043, 'default_optimizer_lr': 5.000000000000001e-05, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 1.5187500715255737}}, 'num_env_steps_sampled': 280000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 280000, 'num_agent_steps_trained': 0}, 'sampler_results': {'episode_reward_max': 30548.17571504754, 'episode_reward_min': -12296.816146656987, 'episode_reward_mean': 2025

In [63]:
algo.restore(checkpoint_path=checkpoint_path)

2023-10-08 17:18:43,635	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-10-08 17:18:43,693	INFO trainable.py:984 -- Restored on 127.0.0.1 from checkpoint: Checkpoint(filesystem=local, path=/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment_simulator/simulated_checkpoints/)


True

In [5]:
my_current_time_step  = datetime(year=1995, month=1, day=1, hour=10) + relativedelta(hours=0)

env_config = {
    "db_params": db_params, 
    "max_episode_steps": 10, 
    "print_output": True,
    "the_current_time_step": my_current_time_step,
}

myEnv = MarketSimulator(env_config)

In [58]:
ray.shutdown()
ray.init(runtime_env={"working_dir": "/Users/mirbilal/Desktop/MobCommission/commissionV2/"})

config = {
    "env": MarketSimulator,
    "env_config": {
        "db_params": db_params, 
        "max_episode_steps": 150, 
        "the_current_time_step": my_current_time_step,
        "print_output": False
    },
    "num_workers": 4,
}

new_algo = ppo.PPO(config=config)
new_algo.restore(checkpoint_path=checkpoint_path)

2023-10-08 17:17:25,813	INFO worker.py:1642 -- Started a local Ray instance.
2023-10-08 17:17:26,278	INFO packaging.py:518 -- Creating a file package for local directory '/Users/mirbilal/Desktop/MobCommission/commissionV2/'.
2023-10-08 17:17:26,606	INFO packaging.py:346 -- Pushing file package 'gcs://_ray_pkg_3b43d8f216855fc5.zip' (10.72MiB) to Ray cluster...
2023-10-08 17:17:26,654	INFO packaging.py:359 -- Successfully pushed file package 'gcs://_ray_pkg_3b43d8f216855fc5.zip'.
2023-10-08 17:17:28,385	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
/Users/mirbilal/Desktop/MobCommission/commissionV2/v2env/lib/python3.10/site-pac

True

In [6]:

import copy

def get_latest_states():
    latest_states = []
    env = myEnv
    env.reset()
    obs = env.state
    latest_states = []  # To store the latest states from each worker
    for _ in range(10):  # Interact with the environment for 10 steps as an example
        action = algo.compute_single_action(obs)  # Replace with your agent's action
        # action = np.array([[664, 831,  51, 988, 280,],
        #             [  1,   0,   1,   1,   0]])
        obs, _, done, _ = env.step(action)
        if done:
            obs = env.reset()
        state = env.state  # Obtain the state from the environment
        latest_states.append(copy.deepcopy(obs))
    return latest_states

latest_states = get_latest_states()


stepping_a [899 530 202  37 318]
[[1.57822450e+02]
 [3.10660832e+00]
 [4.09891726e+02]
 [8.03168974e+03]
 [2.80655342e+04]
 [3.72587265e+03]]
[[  87.40602652 1050.52535324  513.30215289   87.4152659   542.74283225
    87.42130536   86.8         230.41474654]
 [  24.88796476 4085.82720641 2159.98151224   24.89193074 2185.64775091
    24.89968625   25.          800.        ]
 [ 162.1910232  3150.27605446 1631.26782715  162.20821719 1425.56075021
   162.25684778  158.33333333  126.31578947]
 [  37.40892211 2451.738319   1217.04005846   37.41010749 1109.92991571
    37.41508652   37.5         533.33333333]
 [  21.68100633 4404.2458886  2227.88930773   21.69219909 2360.39860704
    21.71976      21.33333333  937.5       ]]
[0]
change_in_shares for index: 1 -265 800.0 2159.9815122390487
change_in_shares for index: 2 -101 126.3157894736842 1631.267827153499
change_in_shares for index: 4 -159 937.5 2227.8893077266875
change_in_shares for index: 0 449 230.41474654377882 513.3021528854566
crack

In [65]:
total_portfolios = []
total_rewards = 0
for astate in latest_states:
    # print(astate)
    total_portfolio_a = 0
    total_portfolio_b = 0
    (stock_state, commodity_state, wallet_state) = astate
    total_portfolio_b = total_portfolio_b + wallet_state[0]
    total_portfolio_a = total_portfolio_a + wallet_state[0]
    # print(stock_state)
    gain = 0
    for a_stck in stock_state:
        total_portfolio_a = total_portfolio_a + a_stck[6]*a_stck[7]
        total_portfolio_b = total_portfolio_b + a_stck[0]*a_stck[7]
        gain = gain + (a_stck[0]*a_stck[7] - a_stck[6]*a_stck[7])
    print(gain)
    print("")
    total_rewards = total_rewards + gain
        
    # print((total_portfolio_a, total_portfolio_b))
    total_portfolios.append((total_portfolio_a, total_portfolio_b))

print("total_rewards", total_rewards)

-2368.5787505716144

-1841.1675964013139

-1277.4526141225815

-1976.5484557908585

-1140.0827290324942

-1007.7464672388451

-1642.0243037815926

-1403.3877523770698

-1057.3095151802063

-1183.6614307732852

total_rewards -14897.959615269861


In [14]:
dividend = 21
divisor = 2

factor = dividend // divisor  # Integer division
remainder = dividend % divisor  # Modulo operation

print(f"Factor (Quotient): {factor}")
print(f"Remainder: {remainder}")

Factor (Quotient): 10
Remainder: 1


In [16]:
ray.shutdown()